In [3]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [4]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [125]:
#  baseurl = 'https://vast-depths-89393.herokuapp.com/'
baseurl = 'https://glacial-hollows-90395.herokuapp.com/'
#baseurl = 'http://127.0.0.1:5000/'

In [126]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [127]:
response.json()

{'signingKey_hex': 'cbdd94a0dafd6c12d181d50a917f9f62cf8c5a62bc136995d28621fb344edb50',
 'traderId': '6',
 'verifyKey_hex': '1a361e497f037a35f44881492795077c544ce79d928f31d3272224bc2adfe9ec'}

In [128]:
url = baseurl + 'createMarket'

mkId = 6
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [129]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [130]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,False,1,1,0,6,6


In [131]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.5678, "quantity":1}

In [132]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [133]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,6,0.5678,1,6


In [134]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [135]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 6,
 'price': 0.5678,
 'quantity': -1,
 'traderId': '6'}

In [136]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 33.95644783973694 seconds.
matched trade: 1 in 30.515822172164917 seconds.
matched trade: 2 in 31.437310218811035 seconds.
matched trade: 3 in 32.816426038742065 seconds.
matched trade: 4 in 35.48816704750061 seconds.
matched trade: 5 in 33.48331618309021 seconds.
matched trade: 6 in 33.80962824821472 seconds.
matched trade: 7 in 41.566872119903564 seconds.
matched trade: 8 in 35.93981099128723 seconds.
matched trade: 9 in 33.282140016555786 seconds.


In [137]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [138]:
response.json()

{'checks': 'False',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 6}

In [139]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [116]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,0.0,1.0
1,1,2,0.1,0.9
2,2,1,0.0,1.0
3,3,1,0.0,1.0
4,4,1,0.0,1.0
5,5,1,0.0,1.0
6,6,1,0.0,1.0


In [117]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [118]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId
0,6,1,0.5678,1,4
1,6,1,0.5678,-1,4
10,6,1,0.5678,1,4
100,6,1,0.5678,1,5
101,6,1,0.5678,-1,5
102,6,1,0.5678,-1,5
103,6,1,0.5678,1,5
104,6,1,0.5678,-1,5
105,6,1,0.5678,1,5
106,6,1,0.5678,1,5


In [119]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 3.2959470748901367 seconds.
unmatched trade: 1 in 3.537036895751953 seconds.
unmatched trade: 2 in 3.3978068828582764 seconds.
unmatched trade: 3 in 3.6110682487487793 seconds.
unmatched trade: 4 in 3.370957136154175 seconds.
unmatched trade: 5 in 3.3296289443969727 seconds.
unmatched trade: 6 in 3.3464930057525635 seconds.
unmatched trade: 7 in 3.342517137527466 seconds.
unmatched trade: 8 in 3.7431910037994385 seconds.
unmatched trade: 9 in 3.396610736846924 seconds.


In [120]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [121]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,6,1,0.44,10,5
1,6,1,0.44,10,6


In [122]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [123]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,6,1,0.5678,0,4
1,6,1,0.5678,0,5
2,6,1,0.5678,0,6


In [124]:
tId

'6'